<a href="https://colab.research.google.com/github/Manos-Mak120/Makrakis-Manos/blob/main/%CE%9C%CE%B1%CE%BA%CF%81%CE%AC%CE%BA%CE%B7%CF%82_%CE%9C%CE%AC%CE%BD%CE%BF%CF%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time
import random

from datetime import datetime

In [96]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    print(f"Error: {response.status_code}")
    return None
  else:
    print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [97]:
def get_teaser_elements_from_articlesL(articlesL):
    articles_dataL = []

    for article in articlesL:
        # Τίτλος
        title_tag = article.find("h3", class_="item-title")
        title = title_tag.text.strip() if title_tag else ""

        # Article URL
        try:
            article_url = title_tag.find("a")["href"] if title_tag else ""
        except:
            article_url = ""

        # Ημερομηνία και Ώρα
        date_str = ""
        time_str = ""
        time_tag = article.find("time")
        if time_tag and time_tag.has_attr("datetime"):
            try:
                datetime_str = time_tag["datetime"]
                date_str = datetime_str.split("T")[0]
                time_str = datetime_str.split("T")[1].split("Z")[0]
            except:
                pass

        # Excerpt (καμία πραγματική περίληψη στο search teaser, οπότε placeholder ή το subtitle αν υπάρχει)
        excerpt = ""
        excerpt_tag = article.find("div", class_="item-text")
        if excerpt_tag:
            excerpt = excerpt_tag.text.strip()

        # Section
        section = ""
        category_tag = article.find("a", class_="item-category")
        if category_tag:
            section = category_tag.text.strip()

        # Author (συνήθως δεν υπάρχει στο teaser)
        author = ""

        articleD = {
            "title": title,
            "article_url": article_url,
            "date": date_str,
            "time": time_str,
            "author": author,
            "excerpt": excerpt,
            "section": section
        }

        articles_dataL.append(articleD)

    return articles_dataL


In [98]:
def scrape_newsbomb_teaser_pages(search_term, start_page_nbr, end_page_nbr):

  all_articles_dataL = []

  print(f"\rScraping started", end="")

  for page_nbr in range(start_page_nbr, end_page_nbr+1):

    url =f"https://www.newsbomb.gr/search?q={search_term}&page={page_nbr}&q={search_term}&section=Articles"

    print(f"\rScraping page {page_nbr}: {url}" + " "*20, end="")

    soup = get_webpage_soup(url)


    articlesL = soup.find("div", {"class": "page-section news-items-section"}).find_all("div", {"class": "card"})

    page_articles_dataL = get_teaser_elements_from_articlesL(articlesL)

    all_articles_dataL.extend(page_articles_dataL)

    delay = random.uniform(1, 3)
    time.sleep(delay)


  df = pd.DataFrame(all_articles_dataL)
  print(f"\rScraping completed. Fetshed {len(df)} articles.")
  return df

In [99]:
trump_df = scrape_newsbomb_teaser_pages("τραμπ", 1, 5)

Scraping page 1: https://www.newsbomb.gr/search?q=τραμπ&page=1&q=τραμπ&section=Articles                    https://www.newsbomb.gr/search?q=τραμπ&page=1&q=τραμπ&section=Articles was scraped successfully
Scraping page 2: https://www.newsbomb.gr/search?q=τραμπ&page=2&q=τραμπ&section=Articles                    https://www.newsbomb.gr/search?q=τραμπ&page=2&q=τραμπ&section=Articles was scraped successfully
Scraping page 3: https://www.newsbomb.gr/search?q=τραμπ&page=3&q=τραμπ&section=Articles                    https://www.newsbomb.gr/search?q=τραμπ&page=3&q=τραμπ&section=Articles was scraped successfully
Scraping page 4: https://www.newsbomb.gr/search?q=τραμπ&page=4&q=τραμπ&section=Articles                    https://www.newsbomb.gr/search?q=τραμπ&page=4&q=τραμπ&section=Articles was scraped successfully
Scraping page 5: https://www.newsbomb.gr/search?q=τραμπ&page=5&q=τραμπ&section=Articles                    https://www.newsbomb.gr/search?q=τραμπ&page=5&q=τραμπ&section=Articles was scrape

In [100]:
trump_df

,title,article_url,date,time,author,excerpt,section
0,Τηλεφωνική επικοινωνία Τραμπ με Ζελένσκι - Η Ο...,/kosmos/story/1647924/oukrania-ipa-tilefoniki-...,,,,,
1,Ο Τραμπ συγχαίρει τον νέο Πάπα Λέοντα 'ΙΔ: «Τι...,/kosmos/story/1647894/o-tramp-sygxairei-ton-ne...,,,,,
2,Επετεύχθη συμφωνία ΗΠΑ και Βρετανίας για τους ...,/kosmos/story/1647867/epeteyxthi-symfonia-ipa-...,,,,,
3,Ο Τραμπ ανακοίνωσε συμφωνία με την Ουκρανία γι...,/kosmos/story/1647860/o-tramp-anakoinose-symfo...,,,,,
4,"Κέισι Μινς: Ινφλουένσερ, επιχειρηματίας και οπ...",/kosmos/story/1647773/keisi-mins-inflouenser-e...,,,,,
...,...,...,...,...,...,...,...
95,Στις κάλπες ο Καναδάς - Ο Τραμπ κρίνει το αποτ...,/kosmos/story/1645068/stis-kalpes-o-kanadas-o-...,,,,,
96,Ο Ντόναλντ Τραμπ υπόσχεται νέες μειώσεις φόρων...,/kosmos/story/1645034/o-donalnt-tramp-yposxeta...,,,,,
97,Τραμπ: Θέλει δωρέαν διελεύσεις για τα αμερικάν...,/kosmos/story/1644900/tramp-thelei-dorean-diel...,,,,,
98,Οι νέοι δασμοί των ΗΠΑ δεν αναμένεται να έχουν...,/kosmos/story/1644868/oi-neoi-dasmoi-ton-ipa-d...,,,,,
